In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=b5d0d92efed87d98625e7d3268384b0ee0fe06499e9450091fd62561a550edd3
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import pandas as pd
import torchtext
import numpy as np
import torch
import torch.nn as nn
import random
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import wandb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/MyDrive/aksharantar_sampled.zip"
!cp "{zip_path}" .
!unzip -q aksharantar_sampled.zip
!rm aksharantar_sampled.zip 

In [ ]:
def get_data(lang):
  train_csv=f"aksharantar_sampled/{lang}/{lang}_train.csv"
  test_csv=f"aksharantar_sampled/{lang}/{lang}_test.csv"
  val_csv=f"aksharantar_sampled/{lang}/{lang}_valid.csv"
  
  return train_csv,test_csv,val_csv



In [ ]:
train,test,val=get_data("hin")

In [ ]:
### For train
input_texts = []
target_texts = []

train_df = pd.read_csv(train, header=None, names=["1", "2"]).astype(str)

# Add all the input and target texts with start sequence and end sequence added to target 
for index, row in train_df.iterrows():
      input_text = row['1']
      target_text = row['2']
      if target_text == '' or input_text == '':
          continue
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)

english_tokens = set()
hindi_tokens = set()

for x,y in zip(input_texts,target_texts):
    for ch in x:
        english_tokens.add(ch)
    for ch in y:
        hindi_tokens.add(ch)
    
english_tokens = sorted(list(english_tokens))
hindi_tokens = sorted(list(hindi_tokens))

eng_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
hin_token_map = dict([(ch,i+1) for i,ch in enumerate(hindi_tokens)])

eng_token_map["<UNK>"]=len(english_tokens)+1
hin_token_map["<UNK>"]=len(hindi_tokens)+1
eng_token_map['<PAD>']=0
hin_token_map['<PAD>']=0

max_eng_len = max([len(i) for i in input_texts])
max_hin_len = max([len(i) for i in target_texts])

In [ ]:
def pre_process(data):
    
    input_texts = []
    target_texts = []
    
    df = pd.read_csv(data, header=None, names=["1", "2"]).astype(str)

    for index, row in df.iterrows():
      input_text = row['1']
      target_text = row['2']
      if target_text == '' or input_text == '':
          continue
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)

    
    a = np.zeros((len(input_texts),max_eng_len+2),dtype="float32")
    b = np.zeros((len(target_texts),max_hin_len+2),dtype="float32")
    
    
    for i,(x,y) in enumerate(zip(input_texts,target_texts)):
        for j,ch in enumerate(x):
            a[i,j] = eng_token_map.get(ch,eng_token_map["<UNK>"])

        for j,ch in enumerate(y):
            b[i,j] = hin_token_map.get(ch,hin_token_map["<UNK>"])
        
      
    return a,b

In [ ]:
trainx, trainy= pre_process(train)
valx, valy= pre_process(val)
testx,testy,= pre_process(test)

In [ ]:
reverse_eng_map = dict([(i,char) for char,i in eng_token_map.items()])
reverse_hin_map = dict([(i,char) for char,i in hin_token_map.items()])

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, input_data, target_data):
        self.input_data = input_data
        self.target_data = target_data
    

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        input_seq = self.input_data[idx]
        target_seq = self.target_data[idx]

        return input_seq, target_seq

def custom_collate(batch):
    input_seqs, target_seqs= zip(*batch)
    input_seqs = torch.from_numpy(np.stack(input_seqs, axis=1))
    target_seqs = torch.from_numpy(np.stack(target_seqs, axis=1))

    return input_seqs, target_seqs


train_dataset = CustomDataset(trainx, trainy)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=custom_collate)

val_dataset = CustomDataset(valx, valy)
#val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)

test_dataset = CustomDataset(testx, testy)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


class Encoder(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size, num_layers, dropout,cell_type):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embed_dim,padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.cell_type=cell_type
        self.fc=nn.Linear(hidden_size*2,hidden_size) # To connect last layer of encoder to first layer of decoder, used when bidirectional=True
                                                     # The hidden_size  of last enocder layer and 1st layer of decoder are same in general
        if cell_type=="LSTM":
            self.rnn=nn.LSTM(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
            self.cell=nn.Linear(hidden_size*2,hidden_size)
        elif cell_type=="GRU":
            self.rnn=nn.GRU(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
        else:
            self.rnn=nn.RNN(embed_dim, hidden_size, num_layers, dropout=dropout,bidirectional=True)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))

        if self.cell_type=="LSTM":
            output, (hidden, cell) = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            cell_concat=torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1)
            cell=torch.tanh(self.fc(cell_concat))


            return output,hidden,cell
        
        elif self.cell_type=="GRU":
            output, hidden = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            return output, hidden
        
        else:
            output, hidden = self.rnn(embedded)
            concated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            hidden = torch.tanh(self.fc(concated))

            return output,hidden

class Attention(nn.Module):
    def __init__(self,hidden_size):
        super().__init__()
        self.enc_hidden_size=hidden_size
        self.dec_hidden_size=hidden_size

        self.fc1 = nn.Linear(self.enc_hidden_size * 2 + self.dec_hidden_size, self.dec_hidden_size)
        self.fc2 = nn.Linear(self.dec_hidden_size, 1, bias=False)
    
    def forward(self,encoder_output,hidden):
        
        batch_size = encoder_output.shape[1]
        source_len = encoder_output.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, source_len, 1)
        
        output = encoder_output.permute(1, 0, 2)
        

        energy = torch.tanh(self.fc1(torch.cat((hidden, output), dim = 2))) 
        

        attention = self.fc2(energy).squeeze(dim=2)
        
        attention_weight = torch.softmax(attention, dim=1)
        
        return attention_weight



class Decoder(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, num_layers, dropout,cell_type,attention):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell_type=cell_type
        self.embedding = nn.Embedding(output_size, embed_dim,padding_idx=0)
        if cell_type=="LSTM":
          self.rnn = nn.LSTM((hidden_size*2)+embed_dim, hidden_size, num_layers,  dropout=dropout)
        elif cell_type=="GRU":
          self.rnn=nn.GRU((hidden_size*2)+embed_dim,hidden_size,num_layers,dropout=dropout,bidirectional=False)
        else:
          self.rnn=nn.RNN(hidden_size*2+embed_dim,hidden_size,num_layers,dropout=dropout)

        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout=nn.Dropout(dropout)
        self.attention=attention

    def forward(self, x,encoder_output,hidden,cell):
        attention = self.attention(encoder_output, hidden).unsqueeze(1)
        output = encoder_output.permute(1, 0, 2)
        context = torch.bmm(attention, output).permute(1, 0, 2)
        x = x.unsqueeze(0)
        embedded = self.dropout(self.embedding(x))

        rnn_input = torch.cat((embedded, context), dim=2)
        if self.cell_type=="LSTM":
          output,(hidden,cell) = self.rnn(rnn_input,(hidden.unsqueeze(0),cell.unsqueeze(0)))
          output= self.fc(output.squeeze(0))
          return output, hidden.squeeze(0),cell.squeeze(0)
        
        elif self.cell_type=="GRU":
          output,hidden = self.rnn(rnn_input,hidden.unsqueeze(0))
          output = output.squeeze(0)
          output= self.fc(output)
          return output, hidden.squeeze(0)
        
        else:
          output,hidden = self.rnn(rnn_input,hidden.unsqueeze(0))
          output = output.squeeze(0)
          output= self.fc(output)
          return output, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
       
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(hin_token_map)

        outputs = torch.zeros(target_len,batch_size, target_vocab_size).to(device)
        if self.encoder.cell_type=="LSTM":
          enc_output ,hidden, cell = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output, hidden, cell = self.decoder(x,enc_output, hidden, cell)
              outputs[t] = output
              top1 = output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1

          return outputs
        
        elif self.encoder.cell_type=="GRU":
          enc_output,hidden = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output,hidden=self.decoder(x,enc_output,hidden,None)
              outputs[t] = output
              top1= output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1
          return outputs
        
        else:
          enc_output,hidden = self.encoder(source)

          x = target[0]

          for t in range(1, target_len):
              output,hidden=self.decoder(x,enc_output,hidden,None)
              outputs[t] = output
              top1= output.argmax(1)
              if random.random() < teacher_forcing_ratio:
                  x = target[t]
              else:
                  x = top1
          return outputs


def build_model(cell = "LSTM",nunits = 64, enc_dec_layers = 1,embed_dim = 128,dropout=0):
    att=Attention(nunits)
    encoder = Encoder(input_size=len(eng_token_map), embed_dim=embed_dim, hidden_size=nunits, num_layers=enc_dec_layers, dropout=dropout,cell_type=cell)
    decoder = Decoder(output_size=len(hin_token_map), embed_dim=embed_dim, hidden_size=nunits, num_layers=enc_dec_layers, dropout=dropout,cell_type=cell,attention=att)
    model = Seq2Seq(encoder, decoder)
    return model


In [ ]:


def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    total_chars = 0
    correct_chars = 0

    for i, (input_seq, target_seq) in enumerate(dataloader):
        input_seq = input_seq.long().to(device)
        target_seq = target_seq.long().to(device)

        optimizer.zero_grad()

        output = model(input_seq, target_seq)
        _, predicted = torch.max(output, dim=2)


        for j in range(predicted.shape[1]):
              predicted_seq = predicted[:, j]
              targets_seq = target_seq[:, j]

              # Find the index of the first EOS token in the sequence (for character & word-level accuracy)
              eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
              if eos_idx.numel() > 0:
                  eos_idx = eos_idx[0][0]
                  predicted_seq = predicted_seq[:eos_idx]
                  targets_seq = targets_seq[:eos_idx]
        
        # reshape for cross-entropy loss
        output_flatten = output[1:].view(-1, output.shape[-1])
        trg_flatten = target_seq[1:].view(-1)

        loss = criterion(output_flatten, trg_flatten)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        predicted_seq=predicted_seq[1:].view(-1)
        targets_seq=targets_seq[1:].view(-1)
     
        correct_chars += torch.sum(predicted_seq == targets_seq).item()
        total_chars += targets_seq.numel()


    return total_loss / len(dataloader), correct_chars/total_chars

In [ ]:

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    total_chars = 0
    correct_chars = 0

    with torch.no_grad():
        for i, (input_seq, target_seq) in enumerate(dataloader):
            input_seq = input_seq.long().to(device)
            target_seq = target_seq.long().to(device)

            output = model(input_seq, target_seq,0)
            _, predicted = torch.max(output, dim=2)


            for j in range(predicted.shape[1]):
                predicted_seq = predicted[:, j]
                targets_seq = target_seq[:, j]

                # Find the index of the first EOS token in the sequence
                eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
                if eos_idx.numel() > 0:
                    eos_idx = eos_idx[0][0]
                    predicted_seq = predicted_seq[:eos_idx]
                    targets_seq = targets_seq[:eos_idx]


            # reshape for cross-entropy loss
            output_flatten = output[1:].view(-1, output.shape[-1])
            trg_flatten = target_seq[1:].view(-1)

            loss = criterion(output_flatten, trg_flatten)

            total_loss += loss.item()

            predicted_seq=predicted_seq[1:].view(-1)
            targets_seq=targets_seq[1:].view(-1)
     
            correct_chars += torch.sum(predicted_seq == targets_seq).item()
            total_chars += targets_seq.numel()

    return total_loss / len(dataloader), correct_chars/total_chars

In [ ]:
'''
N_EPOCHS = 1
best_valid_loss = float('inf')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=custom_collate)

val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)
model=model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=hin_token_map["<PAD>"]) # ignore padding index
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(N_EPOCHS):    
    train_loss,acc = train(model=model, dataloader=train_dataloader, optimizer=optimizer, criterion=criterion,device=device)
    valid_loss,ch = evaluate(model, val_dataloader, criterion,device)


    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    # it's easier to see a change in perplexity between epoch as it's an exponential
    # of the loss, hence the scale of the measure is much bigger
    print(f'Epoch: {epoch+1}')
    print(f'\tTrain Loss: {train_loss}')
    print(f'\t Val. Loss: {valid_loss}')
    print(ch)
    print(acc)
'''

Epoch: 1
	Train Loss: 1.255671155047603
	 Val. Loss: 1.2807335597462952
0.6061717079132295
0.6031660262568044


In [ ]:
def wandb_sweeps():
  config_defaults = {
        'cell': "LSTM",
        'hidden_units': 64,
        'embed_dim': 128,
        'lr':1e-4,
        'dropout':0.1,
        'epochs':20,
        'batch_size':8
  }
  
  wandb.init(config=config_defaults)

  config = wandb.config

  
  cell=config.cell
  hidden_units=config.hidden_units
  embed_dim=config.embed_dim
  lr=config.lr
  epochs=config.epochs
  dropout=config.dropout
  batch_size=config.batch_size


  run_name = f"cell_{cell}_hunit_{hidden_units}_embed_dim_{embed_dim}_lr_{lr}_ep_{epochs}_dropout_{dropout}_bs{batch_size}"
  print(run_name)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

  val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)


  model=build_model(cell = cell,nunits = hidden_units, enc_dec_layers =1,embed_dim = embed_dim,dropout=dropout)
  
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model=model.to(device)

  
  criterion = nn.CrossEntropyLoss(ignore_index=hin_token_map["<PAD>"]) # ignore padding index
  optimizer = optim.Adam(model.parameters(), lr=lr)

  N_EPOCHS = epochs
 
  for epoch in range(N_EPOCHS):    
    train_loss,train_acc = train(model=model, dataloader=train_dataloader, optimizer=optimizer, criterion=criterion,device=device)
    valid_loss,val_acc = evaluate(model, val_dataloader, criterion,device)
    
    wandb.log({"training_acc": train_acc, "validation_accuracy": val_acc, "training_loss": train_loss, "validation_loss": valid_loss, "Epoch": epoch+1})

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        epoch+1, train_loss, valid_loss, train_acc, val_acc))
  
  wandb.run.name = run_name
  wandb.run.save()
  wandb.run.finish()



In [ ]:
sweep_config = {
  "name": "Bayesian Sweep(Attention-2)",
  "method": "bayes",
  "metric":{
  "name": "validation_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': 8,
        's': 2
  },
  "parameters": {
        "cell": {
            "values": [ "LSTM", "GRU", "RNN"]
        },
        "hidden_units": {
            "values": [32,64,256,512]
        },
        "embed_dim": {
            "values": [64,128,256,512]
        }, 
        "lr": {
            "values": [1e-5,1e-4,1e-3]
        },
        "epochs": {
            "values": [10,15,20]
        },
        "dropout": {
            "values": [0,0.1,0.27,0.4]
        },
        "batch_size": {
            "values": [16,32,64]
        }     
    }
}

sweep_id = wandb.sweep(sweep_config,  entity="shashwat_mm19b053", project="Assignment-3")
wandb.agent(sweep_id,project='Assignment-3', function=wandb_sweeps, count=90)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: nye2llmh
Sweep URL: https://wandb.ai/shashwat_mm19b053/Assignment-3/sweeps/nye2llmh


wandb: Agent Starting Run: 4st45xvp with config:
wandb: 	batch_size: 32
wandb: 	cell: GRU
wandb: 	dropout: 0
wandb: 	embed_dim: 128
wandb: 	epochs: 15
wandb: 	hidden_units: 64
wandb: 	lr: 1e-05
wandb: Currently logged in as: shashwat30-04-2001 (shashwat_mm19b053). Use `wandb login --relogin` to force relogin


cell_GRU_hunit_64_embed_dim_128_lr_1e-05_ep_15_dropout_0_bs32
Epoch: 1 	Training Loss: 3.860460 	Validation Loss: 3.468346 	Training Accuracy: 0.063607 	Validation Accuracy: 0.077411
Epoch: 2 	Training Loss: 3.396635 	Validation Loss: 3.294442 	Training Accuracy: 0.085982 	Validation Accuracy: 0.130711
Epoch: 3 	Training Loss: 3.258069 	Validation Loss: 3.186148 	Training Accuracy: 0.120144 	Validation Accuracy: 0.145939
Epoch: 4 	Training Loss: 3.148587 	Validation Loss: 3.078354 	Training Accuracy: 0.154182 	Validation Accuracy: 0.175127
Epoch: 5 	Training Loss: 3.038423 	Validation Loss: 2.966929 	Training Accuracy: 0.172906 	Validation Accuracy: 0.197970
Epoch: 6 	Training Loss: 2.916236 	Validation Loss: 2.831960 	Training Accuracy: 0.204772 	Validation Accuracy: 0.233503
Epoch: 7 	Training Loss: 2.769716 	Validation Loss: 2.677611 	Training Accuracy: 0.233109 	Validation Accuracy: 0.262690
Epoch: 8 	Training Loss: 2.613509 	Validation Loss: 2.535446 	Training Accuracy: 0.276797 	

...

wandb: Agent Starting Run: k4j6vw0a with config:
wandb: 	batch_size: 64
wandb: 	cell: RNN
wandb: 	dropout: 0
wandb: 	embed_dim: 64
wandb: 	epochs: 10
wandb: 	hidden_units: 512
wandb: 	lr: 0.001


cell_RNN_hunit_512_embed_dim_64_lr_0.001_ep_10_dropout_0_bs64
Epoch: 1 	Training Loss: 1.220765 	Validation Loss: 1.136023 	Training Accuracy: 0.602679 	Validation Accuracy: 0.658291
Epoch: 2 	Training Loss: 0.768781 	Validation Loss: 1.068848 	Training Accuracy: 0.751602 	Validation Accuracy: 0.731156
Epoch: 3 	Training Loss: 0.676342 	Validation Loss: 1.077979 	Training Accuracy: 0.773467 	Validation Accuracy: 0.728643
Epoch: 4 	Training Loss: 0.664326 	Validation Loss: 1.210407 	Training Accuracy: 0.786248 	Validation Accuracy: 0.711055
Epoch: 5 	Training Loss: 0.649038 	Validation Loss: 1.035979 	Training Accuracy: 0.790268 	Validation Accuracy: 0.791457
Epoch: 6 	Training Loss: 0.590124 	Validation Loss: 1.026062 	Training Accuracy: 0.793137 	Validation Accuracy: 0.798995
Epoch: 7 	Training Loss: 0.561013 	Validation Loss: 1.046804 	Training Accuracy: 0.791874 	Validation Accuracy: 0.736181
Epoch: 8 	Training Loss: 0.570950 	Validation Loss: 1.039801 	Training Accuracy: 0.811880 	

Epoch,▁▂▃▃▄▅▆▆▇█
training_acc,▁▆▆▇▇▇▇▇██
training_loss,█▃▂▂▂▂▁▁▁▁
validation_accuracy,▁▅▅▄██▅▇▆▇
validation_loss,▅▃▃█▁▁▂▂▁▁
Epoch,10
training_acc,0.83099
training_loss,0.53895
validation_accuracy,0.77889
validation_loss,1.03814


wandb: Agent Starting Run: ctn3e9oy with config:
wandb: 	batch_size: 64
wandb: 	cell: RNN
wandb: 	dropout: 0
wandb: 	embed_dim: 128
wandb: 	epochs: 10
wandb: 	hidden_units: 512
wandb: 	lr: 0.001


cell_RNN_hunit_512_embed_dim_128_lr_0.001_ep_10_dropout_0_bs64
